**Lib**

In [1]:
from google.colab import drive
import pandas as pd
import os
import shutil
from tqdm import tqdm
import random
import PIL
import numpy as np

**Drive**

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


**Path**

In [3]:
metadata_path = '/content/drive/MyDrive/Projects/IS254/Code/metadata'
raw_image_path = '/content/drive/MyDrive/Projects/IS254/Code/raw_image'
image_by_class_path = '/content/drive/MyDrive/Projects/IS254/Code/image_by_class'

os.listdir(image_by_class_path)

['covid', 'normal', 'pneumonia', 'train', 'valid', 'test']

Xoá ảnh lỗi

In [ ]:
!find $data_path/covid -size  0 -print -delete
!find $data_path/pneumonia -size  0 -print -delete
!find $data_path/normal -size  0 -print -delete

find: ‘/covid’: No such file or directory
find: ‘/pneumonia’: No such file or directory
find: ‘/normal’: No such file or directory


Image to maxtrix

In [ ]:
image = PIL.Image.open("/content/00000042_002.png")
image = image.resize((16, 16), PIL.Image.ANTIALIAS)
image_array = np.array(image)

In [ ]:
image_array = [list(row) for row in image_array]

In [ ]:
image_array
print('\n'.join([''.join(['{:4}'.format(item) for item in row]) 
      for row in image_array]))

  21  25  15  28  64 116 154 164 167 162 141  83  33  18  50  46
  22  73  97 121 152 161 126 142 162 127 121 152 119  52  16  39
  74 138 146 156 162 138 119 141 159 122 104 127 157 136  99  60
  98 131 157 170 144  97 100 135 163 116  78  90 150 156 149 124
 114 145 161 167 112  80  81 132 167 110  64  75 135 169 162 125
 131 152 163 158 111  84  81 137 163  90  65  73 126 170 165 143
 143 150 164 151 110  85  95 147 167 116  75  67 119 175 161 153
 141 139 164 145 115  79 100 156 173 112  73  61 111 174 159 150
 129  85 166 139 104  76 104 163 183 118  78  72 102 169 159 144
  93  64 165 135 100  83 115 169 188 151  88  79 105 159 139 114
  67  63 164 124  96  92 119 172 193 169 130  90  98 150 127 100
  42  67 160 116  86  80 115 181 197 175 146  95 103 141 122  95
  14  62 149 120 139 146 151 187 202 179 153 117  88 126 114  92
  13  42 146 162 178 188 193 200 207 197 184 177 156 130  98  64
  15  39 156 172 183 196 204 210 211 203 193 187 180 163  89  37
  16  30 148 176 188 200 

Count images by classes

In [4]:
for label in ['covid', 'normal', 'pneumonia']:
    train = len(os.listdir(image_by_class_path+ '/train' + '/' + label))
    test = len(os.listdir(image_by_class_path+ '/test'+ '/' + label))
    valid = len(os.listdir(image_by_class_path+ '/valid'+ '/' + label))
    print(train, test, valid, train + test + valid)

383 47 49 479
399 49 51 499
400 50 50 500


In [ ]:
for label in ['covid', 'normal', 'pneumonia']:
    files = os.listdir(image_by_class_path+ '/' + label)
    random.shuffle(files)
    total = len(files)
    train_proba = int(total * 0.8)
    test_proba = int(total * 0.1)
    valid_proba = total - train_proba - test_proba
    proba = [0 for i in range(train_proba)] + \
            [1 for i in range(test_proba)] + \
            [2 for i in range(valid_proba)]
    for file, option in tqdm(zip(files, proba), desc=f'Moving {label}'):
        if option == 0:
            src = image_by_class_path + f'/{label}/' + file
            des = image_by_class_path + '/train' + f'/{label}/' + file
            shutil.copy2(src, des)
        if option == 1:
            src = image_by_class_path + f'/{label}/' + file
            des = image_by_class_path + '/test' + f'/{label}/' + file
            shutil.copy2(src, des)
        if option == 2:
            src = image_by_class_path + f'/{label}/' + file
            des = image_by_class_path + '/valid' + f'/{label}/' + file
            shutil.copy2(src, des)

Moving covid: 479it [00:16, 28.28it/s]
Moving normal: 499it [00:19, 26.03it/s]
Moving pneumonia: 500it [00:22, 22.54it/s]


**Read metadata**

In [ ]:
covid_chest_xray = pd.read_csv(f"{metadata_path}/metadata.csv")
chest_xray8 = pd.read_csv(f"{metadata_path}/Data_Entry_2017_v2020.csv")

**Analysis**

covid_chest_xray

In [ ]:
covid_chest_xray.columns

Index(['patientid', 'offset', 'sex', 'age', 'finding', 'RT_PCR_positive',
       'survival', 'intubated', 'intubation_present', 'went_icu', 'in_icu',
       'needed_supplemental_O2', 'extubated', 'temperature', 'pO2_saturation',
       'leukocyte_count', 'neutrophil_count', 'lymphocyte_count', 'view',
       'modality', 'date', 'location', 'folder', 'filename', 'doi', 'url',
       'license', 'clinical_notes', 'other_notes', 'Unnamed: 29'],
      dtype='object')

In [ ]:
covid_chest_xray = covid_chest_xray[['filename', 'finding']]
covid_chest_xray.columns

Index(['filename', 'finding'], dtype='object')

In [ ]:
filenames = covid_chest_xray[covid_chest_xray.finding.str.contains('COVID-19')].filename.values

error = 0
moved = 0
for file in tqdm(filenames, desc='Moving images'):
    src = f"{raw_image_path}/covid_chest_xray/{file}"
    des = f"{image_by_class_path}/covid/{file}"
    try:
        shutil.copy2(src, des)
        moved += 1
    except:
        error += 1



Moving images: 100%|██████████| 584/584 [04:38<00:00,  2.10it/s]


In [ ]:
print(f'Moved {moved} files, Error {error} files')

Moved 560 files, Error 24 files


In [ ]:
#@title
covid_chest_xray.finding.value_counts()

Pneumonia/Viral/COVID-19                   584
todo                                        83
Pneumonia                                   81
Pneumonia/Fungal/Pneumocystis               30
Pneumonia/Bacterial/Streptococcus           22
No Finding                                  22
Tuberculosis                                18
Pneumonia/Viral/SARS                        16
Pneumonia/Lipoid                            13
Pneumonia/Bacterial/Mycoplasma              11
Pneumonia/Viral/MERS-CoV                    10
Pneumonia/Bacterial/Klebsiella              10
Pneumonia/Bacterial/Legionella              10
Pneumonia/Bacterial/Nocardia                 8
Pneumonia/Viral/Varicella                    6
Pneumonia/Viral/Influenza                    5
Pneumonia/Bacterial                          4
Pneumonia/Bacterial/E.Coli                   4
Pneumonia/Viral/Herpes                       3
Pneumonia/Bacterial/Chlamydophila            3
Pneumonia/Viral/Influenza/H1N1               2
Pneumonia/Fun

chest_xray8

In [ ]:
chest_xray8.columns

Index(['Image Index', 'Finding Labels', 'Follow-up #', 'Patient ID',
       'Patient Age', 'Patient Gender', 'View Position', 'OriginalImage[Width',
       'Height]', 'OriginalImagePixelSpacing[x', 'y]'],
      dtype='object')

In [ ]:
chest_xray8 = chest_xray8[['Image Index', 'Finding Labels']]
chest_xray8.columns =  ['filename', 'finding']

In [ ]:
chest_xray8_normal = chest_xray8[chest_xray8.finding.str.strip() == 'No Finding']
chest_xray8_normal[chest_xray8_normal.filename.str.contains('_001')].shape

(8199, 2)

In [ ]:
chest_xray8_pneumonia = chest_xray8[chest_xray8.finding.str.strip() == 'Pneumonia']
for i in range(1, 13):

    if i < 10:
        print(f'file {i}: ', chest_xray8_pneumonia[chest_xray8_pneumonia.filename.str.contains(f'_00{i}')].shape)

    else:
        print(f'file {i}: ', chest_xray8_pneumonia[chest_xray8_pneumonia.filename.str.contains(f'_0{i}')].shape)


file 1:  (40, 2)
file 2:  (34, 2)
file 3:  (16, 2)
file 4:  (15, 2)
file 5:  (16, 2)
file 6:  (11, 2)
file 7:  (17, 2)
file 8:  (8, 2)
file 9:  (6, 2)
file 10:  (5, 2)
file 11:  (7, 2)
file 12:  (4, 2)


In [ ]:
folders = [folder for folder in os.listdir('D:\storage\data\CXR8\images') if folder.find('image')!=-1 and folder.find('tar')==-1]
files = chest_xray8_pneumonia = chest_xray8[chest_xray8.finding.str.strip() == 'Pneumonia'].filename.values

moved = 0
error = 0 
for file in tqdm(files, desc='Moving images'):
    for folder in folders:
        if file in os.listdir('D:/storage/data/CXR8/images/' + folder + '/images'):
            src = 'D:/storage/data/CXR8/images/' + folder + '/images/' + file
            des = 'D:/storage/data/CXR8/pneumonia/'+ file
            try:
                shutil.copy2(src, des)
                moved += 1
            except:
                error += 1

print(f'Moved {moved}, error {error}')

In [ ]:
folders = [folder for folder in os.listdir('D:\storage\data\CXR8\images') if folder.find('image')!=-1 and folder.find('tar')==-1]
files = chest_xray8[chest_xray8.finding.str.strip() == 'No Finding'].filename.sample(500).values
moved = 0
error = 0 
for file in tqdm(files, desc='Moving images'):
    for folder in folders:
        if file in os.listdir('D:/storage/data/CXR8/images/' + folder + '/images'):
            src = 'D:/storage/data/CXR8/images/' + folder + '/images/' + file
            des = 'D:/storage/data/CXR8/normal/'+ file
            try:
                shutil.copy2(src, des)
                moved += 1
            except:
                error += 1

print(f'Moved {moved}, error {error}')